In [148]:
import pandas as pd
import numpy as np
from collections import defaultdict
import pickle

In [151]:
with open('result/2_iter_user_based_interesting_users.pickle', 'rb') as f:
    user_based_prediction = pickle.load(f)
    
with open('result/2_iter_item_based_interesting_users.pickle', 'rb') as f:
    item_based_prediction = pickle.load(f)
    
with open('result/2_iter_most_popular_full.pickle', 'rb') as f:
    most_popular_prediction = pickle.load(f)

with open('users_subsample.pickle', 'rb') as input:
    users_ids = pickle.load(input)
        
with open('result/2_iter_full_actual_purchases.pickle', 'rb') as f:
    actual_purchases = pickle.load(f)

In [8]:
# Наши 2к юзеров
with open('result/users_subsample.pickle', 'rb') as f:
    users = pickle.load(f)
    
products = pd.read_csv('data/products.csv')

In [4]:
#Первая итерация

with open('result/1_iter_ensenble_predictions_models.pickle', 'rb') as f:
    iter1_ensemble_model = pickle.load(f)
    
with open('result/1_iter_ensenble_predictions.pickle', 'rb') as f:
    iter1_ensemble = pickle.load(f)
    
with open('result/1_iter_full_actual_purchases.pickle', 'rb') as f:
    iter1_actual = pickle.load(f)
    
with open('result/1_iter_item_based_interesting_users.pickle', 'rb') as f:
    iter1_item_based = pickle.load(f)
    
with open('result/1_iter_most_popular_full.pickle', 'rb') as f:
    iter1_most_popular = pickle.load(f)
    
with open('result/1_iter_user_based_interesting_users.pickle', 'rb') as f:
    iter1_user_based = pickle.load(f)
    

In [5]:
#Вторая итерация

with open('result/2_iter_ensenble_predictions_models.pickle', 'rb') as f:
    iter2_ensemble_model = pickle.load(f)
    
with open('result/2_iter_ensenble_predictions.pickle', 'rb') as f:
    iter2_ensemble = pickle.load(f)
    
with open('result/2_iter_full_actual_purchases.pickle', 'rb') as f:
    iter2_actual = pickle.load(f)
    
with open('result/2_iter_item_based_interesting_users.pickle', 'rb') as f:
    iter2_item_based = pickle.load(f)
    
with open('result/2_iter_most_popular_full.pickle', 'rb') as f:
    iter2_most_popular = pickle.load(f)
    
with open('result/2_iter_user_based_interesting_users.pickle', 'rb') as f:
    iter2_user_based = pickle.load(f)

<h3> Возьмем троих юзеров <br>
1: Частопокупающий топовые товары (Бананы и органическую фигню))) <br>
2: Юзер-наоборот <br>
3: И Рандомный</h3>

In [293]:

keys = [user for user in users]

user_top = keys[1]
user_ne_top = keys[900]
user_random = keys[1525]
print(f'1)Юзер топ: {user_top}\n\
2)Юзер не топ: {user_ne_top}\n\
3)Рандом: {user_random}')

1)Юзер топ: 199095
2)Юзер не топ: 138451
3)Рандом: 128653


<h2>Первая итерация</h2>

<h3>User top</h3>

In [203]:
df_user_top = pd.DataFrame([])

# Мост попьюлар
df_user_top['most_popular'] = [products[products[' product_id']==product_id]['product_name'].values[0] for product_id in iter1_most_popular[user_top]]

# Item based
df_user_top['item_based'] = [products[products[' product_id']==product_id]['product_name'].values[0] for product_id in iter1_item_based[user_top]]

# User based
df_user_top['user_based'] = [products[products[' product_id']==product_id]['product_name'].values[0] for product_id in iter1_user_based[user_top][:10]]

# Ensemble
df_user_top['ensemble'] = [products[products[' product_id']==product_id]['product_name'].values[0] for product_id in iter1_ensemble[user_top][:10]]

# Actual
df_user_top['actual'] = [products[products[' product_id']==product_id]['product_name'].values[0] for product_id in iter1_actual[user_top][:10]]

df_user_top

,most_popular,item_based,user_based,ensemble,actual
0,Banana,Bag of Organic Bananas,Bag of Organic Bananas,Bag of Organic Bananas,Raspberries
1,Bag of Organic Bananas,Organic Blueberries,Strawberries,Organic Blueberries,Blackberries
2,Organic Strawberries,Strawberries,Raspberries,Strawberries,Clementines
3,Organic Baby Spinach,Raspberries,Hass Avocados,Raspberries,Organic Blueberries
4,Organic Hass Avocado,Hass Avocados,Baby Cucumbers,Hass Avocados,Organic Animal Crackers
5,Organic Avocado,Blackberries,Organic Blueberries,Blackberries,Organic Strawberries
6,Large Lemon,Braeburn Apples,Rainbow Bell Peppers,Braeburn Apples,Seedless Red Grapes
7,Limes,Clementines,Blackberries,Clementines,Hass Avocados
8,Strawberries,"8\"" Vegan Chocolate Cake",Seedless Cucumbers,"8\"" Vegan Chocolate Cake",Bag of Organic Bananas
9,Organic Whole Milk,Pears,Packaged Grape Tomatoes,Pears,Strawberries


In [208]:
one = [products[products[' product_id']==product_id]['product_name'].values[0] for product_id in iter1_item_based[user_top]]
two = [products[products[' product_id']==product_id]['product_name'].values[0] for product_id in iter1_actual[user_top][:10]]
metric = apk(one, two)

In [209]:
print(f'Лучшая моделька: {iter1_ensemble_model[user_top].upper()}\nМетрика = {metric}')

Лучшая моделька: ITEM BASED
Метрика = 0.5991666666666667


<h3>User ne top</h3>

In [289]:
df_user_ne_top = pd.DataFrame([])

# Мост попьюлар
df_user_ne_top['most_popular'] = [products[products[' product_id']==product_id]['product_name'].values[0] for product_id in iter1_most_popular[user_ne_top]]

# Item based
df_user_ne_top['item_based'] = [products[products[' product_id']==product_id]['product_name'].values[0] for product_id in iter1_item_based[user_ne_top]]

# User based
df_user_ne_top['user_based'] = [products[products[' product_id']==product_id]['product_name'].values[0] for product_id in iter1_user_based[user_ne_top][:10]]

# Ensemble
df_user_ne_top['ensemble'] = [products[products[' product_id']==product_id]['product_name'].values[0] for product_id in iter1_ensemble[user_ne_top][:10]]

# Actual
df_user_ne_top['actual'] = [products[products[' product_id']==product_id]['product_name'].values[0] for product_id in iter1_actual[user_ne_top][:10]]

df_user_ne_top

,most_popular,item_based,user_based,ensemble,actual
0,Banana,Cantaloupe Grapefruit Sparkling Water,Organic Half & Half,Organic Half & Half,Gluten Free Dark Chocolate Cocoa Breakfast Bars
1,Bag of Organic Bananas,Pineapple Strawberry Sparkling Water,Grapefruit Sparkling Water,Grapefruit Sparkling Water,Organic Southwestern Burrito
2,Organic Strawberries,Kiwi Sandia Sparkling Water,Organic 2% Reduced Fat Milk,Organic 2% Reduced Fat Milk,Non-Fat Blueberry on the Bottom Greek Yogurt
3,Organic Baby Spinach,Curate Pomme Baya-Apple Berry Sparkling Water,Curate Cherry Lime Sparkling Water,Curate Cherry Lime Sparkling Water,Organic Gluten Free Non-Dairy Beans & Rice Bur...
4,Organic Hass Avocado,"Sparkling Water, Natural Mango Essenced",Lemon Sparkling Water,Lemon Sparkling Water,Pork Carnitas Burrito
5,Organic Avocado,Hummus Roasted Pepper,Passionfruit Sparkling Water,Passionfruit Sparkling Water,Pineapple Strawberry Sparkling Water
6,Large Lemon,Cocoa Roasted Almonds,Total 2% All Natural Greek Strained Yogurt wit...,Total 2% All Natural Greek Strained Yogurt wit...,Peach on the Bottom Nonfat Greek Yogurt
7,Limes,Peach-Pear Sparkling Water,Orange Sparkling Water,Orange Sparkling Water,Cocoa Roasted Almonds
8,Strawberries,Non-Fat Blueberry on the Bottom Greek Yogurt,Kiwi Sandia Sparkling Water,Kiwi Sandia Sparkling Water,Pitted Organic Kalamata Olives To Go!
9,Organic Whole Milk,0% Greek Yogurt Black Cherry on the Bottom,Fridge Pack Cola,Fridge Pack Cola,Non-Fat Greek Yogurt With Strawberries on the ...


In [290]:
one = [products[products[' product_id']==product_id]['product_name'].values[0] for product_id in iter1_item_based[user_ne_top]]
two = [products[products[' product_id']==product_id]['product_name'].values[0] for product_id in iter1_actual[user_ne_top][:10]]
metric = apk(one, two)

In [291]:
print(f'Лучшая моделька: {iter1_ensemble_model[user_ne_top].upper()}\nМетрика = {metric}')

Лучшая моделька: USER BASED
Метрика = 0.10416666666666666


<h3>Random user</h3>

In [300]:
df_user_random = pd.DataFrame([])

# Мост попьюлар
df_user_random['most_popular'] = [products[products[' product_id']==product_id]['product_name'].values[0] for product_id in iter1_most_popular[user_random]]

# Item based
df_user_random['item_based'] = [products[products[' product_id']==product_id]['product_name'].values[0] for product_id in iter1_item_based[user_random]]

# User based
df_user_random['user_based'] = [products[products[' product_id']==product_id]['product_name'].values[0] for product_id in iter1_user_based[user_random][:10]]

# Ensemble
df_user_random['ensemble'] = [products[products[' product_id']==product_id]['product_name'].values[0] for product_id in iter1_ensemble[user_random][:10]]

# Actual
df_user_random['actual'] = [products[products[' product_id']==product_id]['product_name'].values[0] for product_id in iter1_actual[user_random][:10]]

df_user_random

,most_popular,item_based,user_based,ensemble,actual
0,Banana,Organic Vegetable Glycerin Skin Care,Bag of Organic Bananas,Organic Vegetable Glycerin Skin Care,Organic Roma Tomato
1,Bag of Organic Bananas,"Granola, Immortal, Chocolate Almond",Organic Whole Milk,"Granola, Immortal, Chocolate Almond",Russet Potato
2,Organic Strawberries,"Cayenne Pepper, Organic",Organic Strawberries,"Cayenne Pepper, Organic",Purity Farms Ghee Clarified Butter
3,Organic Baby Spinach,Antimo Caputo Flour,Organic D'Anjou Pears,Antimo Caputo Flour,Organic Romaine Leaf
4,Organic Hass Avocado,Grapeseed Natural Skin Care Oil,Organic Hass Avocado,Grapeseed Natural Skin Care Oil,Organic Garlic
5,Organic Avocado,Organic Summer Strawberry Bunny Fruit Snacks,Organic Baby Spinach,Organic Summer Strawberry Bunny Fruit Snacks,Antimo Caputo Flour
6,Large Lemon,Swiss Muesli Original Recipe,Organic Yellow Onion,Swiss Muesli Original Recipe,Gluten Free Covered Pretzels Yogurt
7,Limes,Tamarind Paste,Organic Granny Smith Apple,Tamarind Paste,"Lentil Chips, Spicy Sriracha"
8,Strawberries,Organic Romaine Leaf,Organic Lemon,Organic Romaine Leaf,Organic Ketchup
9,Organic Whole Milk,Organic Plain Whole Milk Yogurt,Organic Bartlett Pear,Organic Plain Whole Milk Yogurt,Organic Unsalted Butter


In [301]:
one = [products[products[' product_id']==product_id]['product_name'].values[0] for product_id in iter1_item_based[user_random]]
two = [products[products[' product_id']==product_id]['product_name'].values[0] for product_id in iter1_actual[user_random][:10]]
metric = apk(one, two)

In [302]:
print(f'Лучшая моделька: {iter1_ensemble_model[user_random].upper()}\nМетрика = {metric}')

Лучшая моделька: ITEM BASED
Метрика = 0.05833333333333333


<hr>

<h2>Вторая итерация</h2>

<h3>User top</h3>

In [217]:
df_user_top = pd.DataFrame([])

# Мост попьюлар
df_user_top['most_popular'] = [products[products[' product_id']==product_id]['product_name'].values[0] for product_id in iter2_most_popular[user_top]]

# Item based
df_user_top['item_based'] = [products[products[' product_id']==product_id]['product_name'].values[0] for product_id in iter2_item_based[user_top]]

# User based
df_user_top['user_based'] = [products[products[' product_id']==product_id]['product_name'].values[0] for product_id in iter2_user_based[user_top][:10]]

# Ensemble
df_user_top['ensemble'] = [products[products[' product_id']==product_id]['product_name'].values[0] for product_id in iter2_ensemble[user_top][:10]]

# Actual
df_user_top['actual'] = [products[products[' product_id']==product_id]['product_name'].values[0] for product_id in iter2_actual[user_top][:10]]

df_user_top

,most_popular,item_based,user_based,ensemble,actual
0,Banana,Organic Blueberries,Bag of Organic Bananas,Organic Blueberries,Blackberries
1,Bag of Organic Bananas,Bag of Organic Bananas,Strawberries,Bag of Organic Bananas,Clementines
2,Organic Strawberries,Strawberries,Raspberries,Strawberries,Braeburn Apples
3,Organic Baby Spinach,Raspberries,Hass Avocados,Raspberries,Organic Blueberries
4,Organic Hass Avocado,Blackberries,Baby Cucumbers,Blackberries,Organic Animal Crackers
5,Organic Avocado,Hass Avocados,Rainbow Bell Peppers,Hass Avocados,"8\"" Vegan Chocolate Cake"
6,Large Lemon,"8\"" Vegan Chocolate Cake",Organic Blueberries,"8\"" Vegan Chocolate Cake",Organic Strawberries
7,Limes,Braeburn Apples,Seedless Cucumbers,Braeburn Apples,Seedless Red Grapes
8,Strawberries,Sleepy Hollow Pinot Noir,Blackberries,Sleepy Hollow Pinot Noir,Bag of Organic Bananas
9,Organic Whole Milk,Pears,Packaged Grape Tomatoes,Pears,Hass Avocados


In [218]:
one = [products[products[' product_id']==product_id]['product_name'].values[0] for product_id in iter2_item_based[user_top]]
two = [products[products[' product_id']==product_id]['product_name'].values[0] for product_id in iter2_actual[user_top][:10]]
metric = apk(one, two)

In [219]:
print(f'Лучшая моделька: {iter2_ensemble_model[user_top].upper()}\nМетрика = {metric}')

Лучшая моделька: ITEM BASED
Метрика = 0.4238888888888888


<h3>User ne top</h3>

In [294]:
df_user_ne_top = pd.DataFrame([])

# Мост попьюлар
df_user_ne_top['most_popular'] = [products[products[' product_id']==product_id]['product_name'].values[0] for product_id in iter2_most_popular[user_ne_top]]

# Item based
df_user_ne_top['item_based'] = [products[products[' product_id']==product_id]['product_name'].values[0] for product_id in iter2_item_based[user_ne_top]]

# User based
df_user_ne_top['user_based'] = [products[products[' product_id']==product_id]['product_name'].values[0] for product_id in iter2_user_based[user_ne_top][:10]]

# Ensemble
df_user_ne_top['ensemble'] = [products[products[' product_id']==product_id]['product_name'].values[0] for product_id in iter2_ensemble[user_ne_top][:10]]

# Actual
df_user_ne_top['actual'] = [products[products[' product_id']==product_id]['product_name'].values[0] for product_id in iter2_actual[user_ne_top][:10]]

df_user_ne_top

,most_popular,item_based,user_based,ensemble,actual
0,Banana,Cantaloupe Grapefruit Sparkling Water,Organic Half & Half,Organic Half & Half,Gluten Free Dark Chocolate Cocoa Breakfast Bars
1,Bag of Organic Bananas,Pineapple Strawberry Sparkling Water,Fridge Pack Cola,Fridge Pack Cola,Organic Southwestern Burrito
2,Organic Strawberries,"Sparkling Water, Natural Mango Essenced",Organic 2% Reduced Fat Milk,Organic 2% Reduced Fat Milk,Non-Fat Blueberry on the Bottom Greek Yogurt
3,Organic Baby Spinach,Kiwi Sandia Sparkling Water,Grapefruit Sparkling Water,Grapefruit Sparkling Water,Organic Gluten Free Non-Dairy Beans & Rice Bur...
4,Organic Hass Avocado,Peach-Pear Sparkling Water,Total 2% All Natural Greek Strained Yogurt wit...,Total 2% All Natural Greek Strained Yogurt wit...,Pork Carnitas Burrito
5,Organic Avocado,Curate Pomme Baya-Apple Berry Sparkling Water,Passionfruit Sparkling Water,Passionfruit Sparkling Water,Pineapple Strawberry Sparkling Water
6,Large Lemon,Cran-Raspberry Flavored Sparkling Water,2% Reduced Fat Milk,2% Reduced Fat Milk,Peach on the Bottom Nonfat Greek Yogurt
7,Limes,Hummus Roasted Pepper,Orange Sparkling Water,Orange Sparkling Water,Cocoa Roasted Almonds
8,Strawberries,Cocoa Roasted Almonds,Curate Cherry Lime Sparkling Water,Curate Cherry Lime Sparkling Water,Pitted Organic Kalamata Olives To Go!
9,Organic Whole Milk,Mango Sparkling Water,Total 2% Lowfat Greek Strained Yogurt With Blu...,Total 2% Lowfat Greek Strained Yogurt With Blu...,Non-Fat Greek Yogurt With Strawberries on the ...


In [295]:
one = [products[products[' product_id']==product_id]['product_name'].values[0] for product_id in iter1_item_based[user_ne_top]]
two = [products[products[' product_id']==product_id]['product_name'].values[0] for product_id in iter1_actual[user_ne_top][:10]]
metric = apk(one, two)

In [296]:
print(f'Лучшая моделька: {iter2_ensemble_model[user_ne_top].upper()}\nМетрика = {metric}')

Лучшая моделька: USER BASED
Метрика = 0.10416666666666666


<h3>Random User</h3>

In [303]:
df_user_random = pd.DataFrame([])

# Мост попьюлар
df_user_random['most_popular'] = [products[products[' product_id']==product_id]['product_name'].values[0] for product_id in iter2_most_popular[user_random]]

# Item based
df_user_random['item_based'] = [products[products[' product_id']==product_id]['product_name'].values[0] for product_id in iter2_item_based[user_random]]

# User based
df_user_random['user_based'] = [products[products[' product_id']==product_id]['product_name'].values[0] for product_id in iter2_user_based[user_random][:10]]

# Ensemble
df_user_random['ensemble'] = [products[products[' product_id']==product_id]['product_name'].values[0] for product_id in iter2_ensemble[user_random][:10]]

# Actual
df_user_random['actual'] = [products[products[' product_id']==product_id]['product_name'].values[0] for product_id in iter2_actual[user_random][:10]]

df_user_random

,most_popular,item_based,user_based,ensemble,actual
0,Banana,Organic Vegetable Glycerin Skin Care,Bag of Organic Bananas,Organic Vegetable Glycerin Skin Care,Organic Roma Tomato
1,Bag of Organic Bananas,"Granola, Immortal, Chocolate Almond",Organic Whole Milk,"Granola, Immortal, Chocolate Almond",Russet Potato
2,Organic Strawberries,"Cayenne Pepper, Organic",Organic Strawberries,"Cayenne Pepper, Organic",Purity Farms Ghee Clarified Butter
3,Organic Baby Spinach,Antimo Caputo Flour,Organic Yellow Onion,Antimo Caputo Flour,Organic Romaine Leaf
4,Organic Hass Avocado,Grapeseed Natural Skin Care Oil,Organic Garlic,Grapeseed Natural Skin Care Oil,Organic Garlic
5,Organic Avocado,Swiss Muesli Original Recipe,Organic Hass Avocado,Swiss Muesli Original Recipe,Antimo Caputo Flour
6,Large Lemon,Organic Summer Strawberry Bunny Fruit Snacks,Carrots,Organic Summer Strawberry Bunny Fruit Snacks,Gluten Free Covered Pretzels Yogurt
7,Limes,Organic Romaine Leaf,Organic Baby Spinach,Organic Romaine Leaf,"Lentil Chips, Spicy Sriracha"
8,Strawberries,Organic Plain Whole Milk Yogurt,Organic Unsalted Butter,Organic Plain Whole Milk Yogurt,Organic Ketchup
9,Organic Whole Milk,Tamarind Paste,Organic Blueberries,Tamarind Paste,Organic Unsalted Butter


In [304]:
one = [products[products[' product_id']==product_id]['product_name'].values[0] for product_id in iter2_item_based[user_random]]
two = [products[products[' product_id']==product_id]['product_name'].values[0] for product_id in iter2_actual[user_random][:10]]
metric = apk(one, two)

In [305]:
print(f'Лучшая моделька: {iter2_ensemble_model[user_random].upper()}\nМетрика = {metric}')

Лучшая моделька: ITEM BASED
Метрика = 0.05833333333333333
